### 【 스크래핑 TAG / SELECTOR 이해 】



[1] 모듈 로딩 및 데이터 준비 <hr>

In [2]:
from bs4 import BeautifulSoup       ## HTML STR ==> Object화 모듈
from urllib.request import urlopen  ## WEB으로부터 Data Loading 모듈

In [3]:
## DATA 준비
HTML_FILE = '../DATA/chap02.html'
HTML_URL = 'https://finance.naver.com'     ## 네이버 증권

In [4]:
## --------------------------------------------------------------------------
## HTML STR 로딩  : 불러오는게 파일인지, url인지에 따라서 다르게 불러와야함
## --------------------------------------------------------------------------
## (1) html 파일
with open(HTML_FILE, mode='r', encoding='utf-8') as f:
    file_data = f.read() ## 전체 다 읽어오기
print(file_data) ## 얘는 String임


## (2) Web 페이지 bytes => str : decoder해서 바꿔줘야함
url_data = urlopen(HTML_URL).read() ## 뒤에 .read() 써야지 데이터만 추출해옴
print(url_data) ## 얘는 binary임 -> string으로 바꿔야함

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>BeautifulSoup 활용</title>
</head>
<body>
    <h1 id="heading">Heading 1</h1>
    <p>Paragraph</p>
    <span class="red">BeautifulSoup Library Examples!</span>
    <div id="link">
        <a class="external_link external_link2" href="www.google.com">google</a>
        <div id="class1">
            <p id="first">class1's first paragraph</p>
            <a class="external_link" href="www.naver.com">naver</a>

            <p id="second">class1's second paragraph</p>
            <a class="internal_link" href="/pages/page1.html">Page1</a>
            <p id="third">class1's third paragraph</p>
        </div>
    </div>
    <div id="text_id2">
        <a class="external_link" href="#heading">Example page</a>
        <p>g</p>
    </div>
    <h1 id="footer">Footer</h1>
</body>
</html>
b'\n\n\t\n\t\n\t\n<html lang=\'ko\'>\n<head>\n\n\n\t\n\t\n\t\t

[2] HTML FILE : HTML STR => BS Object 변환 <hr>
: 파일 불러와서 하는 (1) 부터 진행 

In [5]:
## HTML TAG Object 진행 완료
fileBS = BeautifulSoup(file_data, 'html.parser')

In [6]:
## ===============================================================================
## 원하는 데이터 추출
## ===============================================================================
## find() vs select() : 차이점을 알아야함
## -> find(tag_str, attrs, **kwargs) 메서드     : 제일 첫번째 발견된 태그 가져오기
## -> find_all(tag_str, attrs, **kwargs) 메서드 : 모든 태그 가져오기
## --------------------------------------------------------------------------
## 근데 다 필요한건 아니고, 안에서 딱 하나만 볼려고 하면 어떻게 하냐
## -> select(selector_str) 메서드 : Tag의 id, class 속성을 활용해서
#                                   원하는 태그 모두 추출
## -> select_one(selector_str) 메서드 : Tag의 id, class 속성을 활용해서
#                                       한개 태그 모두 추출
## ===============================================================================

In [7]:
## => class1's first paragraph 텍스트 추출
ret1 = fileBS.find('p', attrs={'id':'first'})  
ret2 = fileBS.select('#first')                 ## id가 first는 하나밖에 없어서 바로나옴

print(f'ret1 => {ret1}')
print(f'ret1 => {ret2}')

ret1 => <p id="first">class1's first paragraph</p>
ret1 => [<p id="first">class1's first paragraph</p>]


In [8]:
## => naver 텍스트 추출 [내풀이]
## find()
ret1 = fileBS.find('a', attrs={'class':"exteranl_link"})
print(ret1)

# ## select()
# ret2 = fileBS.select("a") 
# ret2

None


In [9]:
## => naver 텍스트 추출 [강사님]
## find 기반
ret1 = fileBS.find('div')
print(ret1)

ret2 = ret1.find_all('a') ## 모든 a가 나옴
print('ret2 :', ret2)

ret2 = ret1.find('a', attrs={'href':"www.naver.com"})
print('ret2 :', ret2)

ret2 = ret1.find('a', href = "www.naver.com")   ## kwargs 파라미터
print('ret2 :', ret2)

ret2 = ret1.find(string='naver')
print('ret2 :', ret2)

## - selector 기반
# div에 클래스명 안에 a 태그에 ext~~ 를 선택하는 식으로
fileBS.select_one('div#class1>a.exteranl_link') 


<div id="link">
<a class="external_link external_link2" href="www.google.com">google</a>
<div id="class1">
<p id="first">class1's first paragraph</p>
<a class="external_link" href="www.naver.com">naver</a>
<p id="second">class1's second paragraph</p>
<a class="internal_link" href="/pages/page1.html">Page1</a>
<p id="third">class1's third paragraph</p>
</div>
</div>
ret2 : [<a class="external_link external_link2" href="www.google.com">google</a>, <a class="external_link" href="www.naver.com">naver</a>, <a class="internal_link" href="/pages/page1.html">Page1</a>]
ret2 : <a class="external_link" href="www.naver.com">naver</a>
ret2 : <a class="external_link" href="www.naver.com">naver</a>
ret2 : naver


In [10]:
## 항목에서 태그 속성이 중복된 경우 추출 하기
html_str="""
<html>
<body>
<div id="main-goods" role="page">
  <h1>과일과 야채</h1>
  <ul id="fr-list">
    <li class="red green" data-lo="ko">사과</li>
    <li class="purple" data-lo="us">포도</li>
    <li class="yellow" data-lo="us">레몬</li>
    <li class="yellow" data-lo="ko">오렌지</li>
  </ul>
  <ul id="ve-list">
    <li class="white green" data-lo="ko">무</li>
    <li class="red green" data-lo="us">파프리카</li>
    <li class="black" data-lo="ko">가지</li>
    <li class="black" data-lo="us">아보카도</li>
    <li class="white" data-lo="cn">연근</li>
  </ul>
</div>
</body>
</html>
"""

## => 아보카도 추출 -------------------------------------
## HTML TAG Object 진행 완료
strBS = BeautifulSoup(html_str, 'html.parser')


# CSS 선택자로 추출
## li:nth-of-type(번호) 1번부터 시작. 같은 종류의 태그
## li:nth-of-child(번호) 1번부터 시작. 같은 종류의 태그 아닐 수 있음. 자식이면 됨 !! 
print(strBS.select_one("li:nth-of-type(8)"))                ## 부모 태그가 나눠져 있어서 자식이 8개 없음
print(strBS.select_one("#ve-list > li:nth-of-type(4)").text) ## 타입이면 <a> 같은 거에서 카운팅하고
print(strBS.select_one("#ve-list > li:nth-child(4)").text)  ## child면 하위에서 순서대로 카운팅한다.
print(strBS.select("#ve-list > li[data-lo='us']")[1].text)
print(strBS.select("#ve-list > li.black")[1].text)




None
아보카도
아보카도
아보카도
아보카도


In [15]:
# find 메서드로 추출 : 여러개 속성 조합 시 dict 묶어서 처리
cond = {'data-lo':'us', 'class':'black'}
print(strBS.find("li", cond).text)

# find 메서드를 연속적으로 사용
print(strBS.find(id='ve-list').find('li', cond).text) ## 부모 데이터 빼고, 안에서 자식을 다시 선택하는 방식

아보카도
아보카도


In [16]:
# 파프리카 태그 찾기
paprika = strBS.find("li", string = "파프리카")
print(f'* avpaprika Tag => {paprika}')

# 파프리카 태그가 속한 부모 찾기 : find_parent()
# -> 가장 가까운 ul 태그 찾음
parent_ul = paprika.find_parent("ul")
print(f'* paprika 부모 Tag -> {parent_ul}')
print(f'* paprika 부모 Tag id -> {parent_ul["id"]}')

# 파프리카 다음 요소 태그 찾기 : find_next()
# -> 순서 : 위에서 아래, 왼쪽 -> 오른쪾
# -> 다음 li 태그 찾음
next_li = paprika.find_next("li")
print(f'* paprika 다음 Tag : {next_li.text}, {next_li}')

next_us = paprika.find_next("li", {"data-lo" : "us"})
print(f'* paprika 다음 data-lo가 us인 Tag : {next_us.text}, {next_us}')

# 파프리카 형재 요소들 중 태그 찾기 : find_next_sibling()
# -> 형제 요소들 중 찾기
next_sibling = paprika.find_next_sibling("li")
print(f'*paprika 형제 중 다음 Tag : {next_sibling.text}')

* avpaprika Tag => <li class="red green" data-lo="us">파프리카</li>
* paprika 부모 Tag -> <ul id="ve-list">
<li class="white green" data-lo="ko">무</li>
<li class="red green" data-lo="us">파프리카</li>
<li class="black" data-lo="ko">가지</li>
<li class="black" data-lo="us">아보카도</li>
<li class="white" data-lo="cn">연근</li>
</ul>
* paprika 부모 Tag id -> ve-list
* paprika 다음 Tag : 가지, <li class="black" data-lo="ko">가지</li>
* paprika 다음 data-lo가 us인 Tag : 아보카도, <li class="black" data-lo="us">아보카도</li>
*paprika 형제 중 다음 Tag : 가지


In [ ]:
## =========================================
## 조건에 맞는 과일 & 채소 추출
## =========================================
## HTML TAG Object 진행 완료 
strBS = BeautifulSoup(html_str, 'html.parser')

## (1) 미국산 과일
fr_list = strBS.find("ul", id="fr-list")
first_us_fruit = fr_list.find("li", {"data-lo": "us"})
print("[find] 미국산 과일:", first_us_fruit.text)  

## (2) 연근의 부모 태그 찾기 
root = strBS.find("li", string="연근")
parent_ul = root.find_parent("ul")
print("[find_parent] 연근 부모 ul id:", parent_ul["id"])  

## (3) 레몬 다음 과일
lemon   = strBS.find("li", string="레몬")
next_li = lemon.find_next("li")
print("[find_next] 레몬 다음 li:", next_li.text)   

## (3) 레몬 이전 한국산 과일
pre_ko_fruits = lemon.find_previous("li", {"data-lo": "ko"})
print("[find_previous] 레몬 이전 한국산 과일:", pre_ko_fruits.text)  
